In [ ]:
!pip install transformers accelerate peft bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login
import torch
import os

hf_token = os.environ.get("HF_TOKEN")

login(hf_token)

tokenizer = AutoTokenizer.from_pretrained("./lora_configs")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Change to False for 8-bit
    bnb_4bit_quant_type="nf4",  # or "fp4"
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",  # spreads across available devices
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, "./lora_configs")

print("Prompt Enhancer CLI — type 'exit' to quit\n")
while True:
    user_input = input("Messy prompt: ")
    if user_input.lower() == "exit":
        break

    prompt = f"Enhance this prompt: {user_input}"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=500)
    enhanced = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("\n✨ Enhanced prompt:\n", enhanced, "\n")
